In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.optim as optim


from zenml.pipelines import pipeline
from zenml.steps import step, Output

import optuna
from optuna.trial import TrialState

device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
@pipeline
def pytorch_hpo_pipeline(
    load_data,
    run_hpo,
    train_test,
):
    """A `pipeline` to load data, load model, and train/evaluate the model."""
    train_dataloader, test_dataloader = load_data()
    best_hparams = run_hpo(train_dataloader, test_dataloader)
    train_test(best_hparams, train_dataloader, test_dataloader)


In [11]:
def get_mnist():
    
    batch_size = 64

    # Download training data from open datasets.
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader

@step
def load_data() -> Output(
    train_dataloader=DataLoader, test_dataloader=DataLoader
):
    """A `step` to load the Fashion MNIST dataset as a tuple of torch Datasets."""

    train_dataloader, test_dataloader = get_mnist()

    return train_dataloader, test_dataloader

In [12]:
def define_hpo_model(trial):
    CLASSES = 10
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    in_features = 28 * 28
    layers.append(nn.Flatten())
    
    for i in range(n_layers):
        out_features = trial.suggest_int(f"n_units_l{i}", 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float(f"dropout_l{i}", 0.2, 0.5)
        layers.append(nn.Dropout(p))
        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

def objective(trial, train_dataloader, test_dataloader):

    # Generate model
    model = define_hpo_model(trial)
    
    # Train model
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer_name = trial.suggest_categorical("optimizer_name", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    size = len(train_dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test_accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return test_accuracy


@step
def run_hpo(
    train_dataloader: DataLoader, 
    test_dataloader: DataLoader
) -> dict:
    
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, train_dataloader, test_dataloader), n_trials=30, timeout=600)
    
    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    best = trial.params

    print("Found best hyperparam dict from optimization work")
    print(best)
    
    return best

In [13]:
def define_model(hparam:dict):
    CLASSES = 10
    layers = []
    in_features = 28 * 28
    layers.append(nn.Flatten())
    
    print(hparam)
    
    for i in range(hparam['n_layers']):
        out_features = hparam[f"n_units_l{i}"]
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = hparam[f"dropout_l{i}"]
        layers.append(nn.Dropout(p))
        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    """A function to train a model for one epoch."""
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    """A function to test a model on the validation / test dataset."""
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    test_accuracy = 100*correct
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return test_accuracy

In [15]:
@step
def train_test(
    best_hparams: dict,
    train_dataloader: DataLoader, 
    test_dataloader: DataLoader
) -> Output(trained_model=nn.Module, test_acc=float):
    """A `step` to train and evaluate a torch model on given dataloaders."""
    
    epochs = 5
    
    model = define_model(best_hparams)
    
    print("Constructing optimized model:")
    print(model)

    print("Using optimized hyperparams")
    print(best_hparams)

    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    
    lr = best_hparams['lr']
    optimizer_name = best_hparams['optimizer_name']
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    test_acc = 0
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test_acc = test(test_dataloader, model, loss_fn)
    print("Done!")

    return model, test_acc

In [16]:
pytorch_hpo_pipeline(
    load_data=load_data(),
    run_hpo=run_hpo(),
    train_test=train_test(),
).run(unlisted=True)

Unable to find ZenML repository in your current working directory (/home/dnth/Desktop/zenml-optuna) or any parent directories. If you want to use an existing repository which is in a different location, set the environment variable 'ZENML_REPOSITORY_PATH'. If you want to create a new repository, run zenml init.
Running without an active repository root.
Running unlisted pipeline on stack default (caching enabled)
Step load_data has started.
Step load_data has finished in 0.122s.
Step run_hpo has started.


[I 2022-11-28 15:24:05,820] A new study created in memory with name: no-name-c531756c-2561-4fa8-8a72-4b8b3737249a
[I 2022-11-28 15:24:12,546] Trial 0 finished with value: 71.55 and parameters: {'n_layers': 3, 'n_units_l0': 37, 'dropout_l0': 0.2825931202872253, 'n_units_l1': 97, 'dropout_l1': 0.3712839442432332, 'n_units_l2': 28, 'dropout_l2': 0.29779249195626956, 'optimizer_name': 'Adam', 'lr': 0.011292325923164617}. Best is trial 0 with value: 71.55.


Test Error: 
 Accuracy: 71.5%, Avg loss: 0.696968 



[I 2022-11-28 15:24:17,413] Trial 1 finished with value: 66.18 and parameters: {'n_layers': 2, 'n_units_l0': 64, 'dropout_l0': 0.3123391111812263, 'n_units_l1': 30, 'dropout_l1': 0.27063224887186577, 'optimizer_name': 'Adam', 'lr': 6.271506458338351e-05}. Best is trial 0 with value: 71.55.


Test Error: 
 Accuracy: 66.2%, Avg loss: 1.107685 



...

Test Error: 
 Accuracy: 81.5%, Avg loss: 0.494065 



[I 2022-11-28 15:26:22,402] Trial 28 finished with value: 75.33999999999999 and parameters: {'n_layers': 1, 'n_units_l0': 89, 'dropout_l0': 0.23703655843983137, 'optimizer_name': 'RMSprop', 'lr': 0.018553618102427272}. Best is trial 25 with value: 83.48.


Test Error: 
 Accuracy: 75.3%, Avg loss: 0.675442 



[I 2022-11-28 15:26:26,857] Trial 29 finished with value: 68.52000000000001 and parameters: {'n_layers': 1, 'n_units_l0': 47, 'dropout_l0': 0.2991821400337635, 'optimizer_name': 'RMSprop', 'lr': 0.034249674851051216}. Best is trial 25 with value: 83.48.


Test Error: 
 Accuracy: 68.5%, Avg loss: 0.891046 

Study statistics: 
  Number of finished trials:  30
  Number of pruned trials:  0
  Number of complete trials:  30
Best trial:
  Value:  83.48
  Params: 
    n_layers: 1
    n_units_l0: 73
    dropout_l0: 0.24536934603161373
    optimizer_name: RMSprop
    lr: 0.002076783936115092
Found best hyperparam dict from optimization work
{'n_layers': 1, 'n_units_l0': 73, 'dropout_l0': 0.24536934603161373, 'optimizer_name': 'RMSprop', 'lr': 0.002076783936115092}
Step run_hpo has finished in 2m21s.
Step train_test has started.
{'n_layers': 1, 'n_units_l0': 73, 'dropout_l0': 0.24536934603161373, 'optimizer_name': 'RMSprop', 'lr': 0.002076783936115092}
Constructing optimized model:
Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=73, bias=True)
  (2): ReLU()
  (3): Dropout(p=0.24536934603161373, inplace=False)
  (4): Linear(in_features=73, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)
Using 